# Trading Statistics Service - Historical Performance Metrics

Analyze trading statistics across multiple time periods (YTD, 1M, 3M, 6M, 1Y) including price, volume, and valuation metrics.

## Overview

The **Trading Statistics Service** provides multi-period historical data:

- **Time Periods**: YTD, 1 Month, 3 Months, 6 Months, 1 Year
- **Price Data**: OHLC (Open, High, Low, Close), average price
- **Volume Metrics**: Total volume, total value, average daily value
- **Valuation**: P/E, P/B, market cap, dividend yield
- **Risk**: Beta coefficient
- **Liquidity**: Turnover ratio

### When to Use

- Historical performance analysis
- Trend identification
- Volatility assessment
- Comparative period analysis
- Technical trading strategies

In [ ]:
!pip install settfex

In [8]:
import asyncio
from settfex.services.set import get_trading_stats
import pandas as pd
import matplotlib.pyplot as plt

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [29]:
# Fetch trading statistics
stats = await get_trading_stats("MINT")

print(f"Trading Statistics for MINT ({len(stats)} periods)\n")

for stat in stats:
    print(f"Period: {stat.period}")
    print(f"  Date: {stat.date}")
    print(f"  Close: {stat.close:.2f} THB ({stat.percent_change:+.2f}%)")
    print(f"  High: {stat.high:.2f}, Low: {stat.low:.2f}")
    print(f"  Volume: {stat.total_volume:,}")
    print(f"  P/E: {stat.pe}, Market Cap: {stat.market_cap:,.0f} THB")
    print()

Trading Statistics for MINT (3 periods)

Period: YTD
  Date: 2025-10-03 00:00:00+07:00
  Close: 23.00 THB (-11.54%)
  High: 29.50, Low: 21.70
  Volume: 3,365,073,549.0
  P/E: 17.91, Market Cap: 130,409,470,471 THB

Period: 2024
  Date: 2024-12-30 00:00:00+07:00
  Close: 26.00 THB (-11.86%)
  High: 34.00, Low: 23.80
  Volume: 3,864,045,824.0
  P/E: 28.89, Market Cap: 147,419,401,402 THB

Period: 2023
  Date: 2023-12-28 00:00:00+07:00
  Close: 29.50 THB (-8.53%)
  High: 35.00, Low: 25.75
  Volume: 4,250,693,728.0
  P/E: 26.06, Market Cap: 165,076,043,154 THB



## Advanced Usage - Performance Comparison

In [10]:
# Convert to DataFrame for analysis
df = pd.DataFrame([
    {
        "period": s.period,
        "close": s.close,
        "change_pct": s.percent_change,
        "volume": s.total_volume,
        "avg_value": s.average_value,
        "pe": s.pe,
        "pbv": s.pbv,
        "beta": s.beta
    }
    for s in stats
])

print("Performance by Period:\n")
print(df[['period', 'close', 'change_pct', 'pe', 'beta']].to_string(index=False))

Performance by Period:

period  close  change_pct    pe  beta
   YTD   23.0  -11.538462 17.91  1.08
  2024   26.0  -11.864407 28.89  1.19
  2023   29.5   -8.527132 26.06  1.04


## Use Case: Multi-Stock Performance Dashboard

In [ ]:
async def performance_dashboard(symbols: list[str], period: str = "YTD"):
    """
    Build performance dashboard for specific period.
    Available periods: "YTD", "2024", "2023", etc.
    """
    tasks = [get_trading_stats(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    dashboard = []
    for symbol, stats in zip(symbols, results):
        if isinstance(stats, Exception):
            continue
        
        # Find specific period
        period_stat = next((s for s in stats if s.period == period), None)
        if period_stat:
            dashboard.append({
                "symbol": symbol,
                "close": period_stat.close,
                "change_pct": period_stat.percent_change,
                "volume": period_stat.total_volume,
                "pe": period_stat.pe,
                "beta": period_stat.beta,
                "turnover": period_stat.turnover_ratio
            })
    
    df = pd.DataFrame(dashboard).sort_values('change_pct', ascending=False)
    return df

# Build dashboard
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL", "SCB", "MINT"]
dashboard_ytd = await performance_dashboard(symbols, period="YTD")


In [27]:
print(f"YTD Performance Dashboard:\n")
print(dashboard_ytd.to_string(index=False))

YTD Performance Dashboard:

symbol  close  change_pct       volume    pe  beta  turnover
   SCB 127.00    8.085106 1698678204.0  8.92  0.50 50.449186
 KBANK 165.50    6.430868 2368875681.0  8.05  0.44 99.980927
   PTT  32.25    1.574803 9076955456.0 12.96  0.55 31.778723
   BBL 148.50   -1.655629 1489230475.0  5.99  0.45 78.017446
  MINT  23.00  -11.538462 3365073549.0 17.91  1.08 59.348981
 CPALL  48.00  -13.901345 6802733754.0 15.78  1.01 75.728120
   AOT  43.50  -26.890756 6656459108.0 33.53  1.39 46.595260


## Use Case: Volatility Analysis

In [12]:
# Compare volatility across periods
volatility_data = []
for stat in stats:
    if stat.high and stat.low and stat.close:
        price_range = ((stat.high - stat.low) / stat.close) * 100
        volatility_data.append({
            "period": stat.period,
            "price_range_pct": price_range,
            "beta": stat.beta
        })

vol_df = pd.DataFrame(volatility_data)
print("Volatility by Period:\n")
print(vol_df.to_string(index=False))

Volatility by Period:

period  price_range_pct  beta
   YTD        33.913043  1.08
  2024        39.230769  1.19
  2023        31.355932  1.04


## Next Steps

- **[Highlight Data](02_highlight_data.ipynb)** - Current market metrics
- **[Price Performance](10_price_performance.ipynb)** - Comparative performance

**Documentation**: `/docs/settfex/services/set/trading_stat.md`